In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[                     
                      'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)',#13
    
                      'Fe Precursor Ratio',#14
                      'Zn Precursor Ratio',#15
                      '2-Methylimidazole Precursor Ratio',#16
                      'Additive (Other) Precursor Ratio',#17
                      'ZIF Preparation Temperature (℃)',#18
                      'ZIF Preparation Time (h)',#19
                      'One-Pot Method',#20
                      'Post-Adsorb Method',#21
                      'Carbon-Derivative Adsorb Method',#22
                      'Pyrolysis Temperature (℃)',#23
                      'Pyrolysis Time (h)',#24
                      'Temperature Rising Rate (℃ min-1)',#25
                      'Seconde Pyrolysis',#26
                      'Acid Wash'#27
                        ]]

In [ ]:
print(np.mean(raw_data))
print(np.std(raw_data))

In [ ]:
mat=raw_data.corr(method='spearman')#kendall #pearson
print(type(mat))

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
f, ax= plt.subplots(figsize = (14, 10))
sns.set(font_scale=0.55)
sns.heatmap(mat, 
                annot=True, vmax=1,vmin = 0, xticklabels= True, yticklabels= True, square=True, cmap="gist_heat_r")

In [ ]:
chemical_properies=raw_data.iloc[:,0:15]
synthesis=raw_data.iloc[:,15:29]

In [ ]:
CL_78=fdata.loc[:,[                     
                      'HW_0.78'
]]
CL_80=fdata.loc[:,[                     
                      'HW_0.8'
]]
CL_82=fdata.loc[:,[                     
                      'HW_0.82'
]]
CL_003=fdata.loc[:,[                     
                      'PYMT_0.03'
]]
CL_002=fdata.loc[:,[                     
                      'PYMT_0.02'
]]

In [ ]:
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total
from sklearn.decomposition import PCA

In [ ]:
pca_chem_3 = PCA(n_components=3)
chem_3 = pca_chem_3.fit_transform(chemical_properies)     
print(pca_chem_3.explained_variance_ratio_)

pca_synthesis_3 = PCA(n_components=3)
synthesis_3 = pca_synthesis_3.fit_transform(synthesis)   
print(pca_synthesis_3.explained_variance_ratio_)

rpca_chem_3 = PCA(n_components=3,svd_solver='randomized')
chem_3_r = rpca_chem_3.fit_transform(chemical_properies)    
print(rpca_chem_3.explained_variance_ratio_)

rpca_synthesis_3 = PCA(n_components=3,svd_solver='randomized')
synthesis_3_r = rpca_synthesis_3.fit_transform(synthesis)     
print(rpca_synthesis_3.explained_variance_ratio_)

pca_chem_2 = PCA(n_components=2)
chem_2 = pca_chem_2.fit_transform(chemical_properies)     
print(pca_chem_2.explained_variance_ratio_)

pca_synthesis_2 = PCA(n_components=2)
synthesis_2 = pca_synthesis_2.fit_transform(synthesis)   
print(pca_synthesis_2.explained_variance_ratio_)

rpca_chem_2 = PCA(n_components=2,svd_solver='randomized')
chem_2_r = rpca_chem_2.fit_transform(chemical_properies)    
print(rpca_chem_2.explained_variance_ratio_)

rpca_synthesis_2 = PCA(n_components=2,svd_solver='randomized')
synthesis_2_r = rpca_synthesis_2.fit_transform(synthesis)     
print(rpca_synthesis_2.explained_variance_ratio_)



print(summarize(pca_chem_3.explained_variance_ratio_))
print(summarize(pca_synthesis_3.explained_variance_ratio_))
print(summarize(rpca_chem_3.explained_variance_ratio_))
print(summarize(rpca_synthesis_3.explained_variance_ratio_))

print(summarize(pca_chem_2.explained_variance_ratio_))
print(summarize(pca_synthesis_2.explained_variance_ratio_))
print(summarize(rpca_chem_2.explained_variance_ratio_))
print(summarize(rpca_synthesis_2.explained_variance_ratio_))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def plot_3d_PCA(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    ax=Axes3D(fig)
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        z = dataset[:, 2][cl == i]
        ax.scatter(x,y,z,c=colors[i])
        ax.set_zlabel('Third Principal Component')
    # 设置图例，0-9用不同颜色表示
    plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # 设置坐标标签
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    # 设置标题
    plt.title("PCA Scatter Plot")
    plt.show()
def plot_2d_PCA(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        plt.scatter(x,y,c=colors[i])
    # 设置图例，0-9用不同颜色表示
    plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # 设置坐标标签
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    # 设置标题
    plt.title("PCA Scatter Plot")
    # 显示图形
    plt.show()

In [ ]:
plot_3d_PCA(chem_3,CL_78)
plot_3d_PCA(chem_3,CL_80)
plot_3d_PCA(chem_3,CL_82)

In [ ]:
plot_3d_PCA(chem_3_r,CL_78)
plot_3d_PCA(chem_3_r,CL_80)
plot_3d_PCA(chem_3_r,CL_82)

In [ ]:
plot_3d_PCA(synthesis_3,CL_003)
plot_3d_PCA(synthesis_3,CL_002)

In [ ]:
plot_3d_PCA(synthesis_3_r,CL_003)
plot_3d_PCA(synthesis_3_r,CL_002)

In [ ]:
plot_2d_PCA(chem_3,CL_78)
plot_2d_PCA(chem_3,CL_80)
plot_2d_PCA(chem_3,CL_82)

In [ ]:
plot_2d_PCA(chem_3_r,CL_78)
plot_2d_PCA(chem_3_r,CL_80)
plot_2d_PCA(chem_3_r,CL_82)

In [ ]:
plot_2d_PCA(synthesis_3,CL_003)
plot_2d_PCA(synthesis_3,CL_002)

In [ ]:
plot_2d_PCA(synthesis_3_r,CL_003)
plot_2d_PCA(synthesis_3_r,CL_002)